In [ ]:
import os
import random as random
from tqdm.notebook import tqdm
from elasticsearch import Elasticsearch
import pandas as pd
import json

In [ ]:
client = Elasticsearch(
    os.environ["ELASTIC_PIPELINE_HOST"],
    basic_auth=(
        os.environ["ELASTIC_PIPELINE_USERNAME"],
        os.environ["ELASTIC_PIPELINE_PASSWORD"],
    ),
)

In [ ]:
bad_image_matches = [
    {"source_id": "fdgrjrwb", "target_id": "v75jmdmc"},
    {"source_id": "dwhuv3ph", "target_id": "cg7hzgv8"},
]

In [ ]:
image_id = bad_image_matches[0]["source_id"]

In [ ]:
similar_images = client.search(
    index="images-indexed-2022-07-04",
    query={
        "more_like_this": {
            "fields": [
                "state.inferredData.lshEncodedFeatures",
                "state.inferredData.palette"
            ],
            "like": [
                {
                    "_index": "images-indexed-2022-07-04",
                    "_id": image_id,
                }
            ],
            "min_term_freq": 1,
            "min_doc_freq": 1,
            "max_query_terms": 1000,
            "minimum_should_match": "1",
        }
    },
    size=6,
    _source=["_id"],
)
results = similar_images["hits"]["hits"]

In [ ]:
match = bad_image_matches[0]

response = client.explain(
    index="images-indexed-2022-07-04",
    id=match["target_id"],
    query={
        "more_like_this": {
            "fields": [
                "state.inferredData.lshEncodedFeatures",
                "state.inferredData.palette"
            ],
            "like": [
                {
                    "_index": "images-indexed-2022-07-04",
                    "_id": match["source_id"],
                }
            ],
            "min_term_freq": 1,
            "min_doc_freq": 1,
            "max_query_terms": 1000,
            "minimum_should_match": "1",
        }
    },
    _source=["_id"],
)
explanation = json.dumps(response['explanation'], indent=2)
print(explanation)